----------------------------------------
### Préparation de l'environnement

* Importe la donnée Carbosense curated
* Import les packages pythons

In [ ]:
!git lfs pull

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy as sc
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import plot_tree
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error as mae

-----
### Lire la donnée dans une DataFrame pandas

In [ ]:
DATA = '../data/carbosense-curated/CarboSense-October2017_curated.csv'

In [ ]:
df = pd.read_csv(DATA)
df['Timestamp'] = pd.to_datetime(df['Timestamp'])
df = df.dropna()
df.head()

In [ ]:
df.Location_Name.unique()

-----

Pour les experts métier, nous savons qu'il existe une relation entre le niveau de CO2, la temperature, l'humidity, l'heure et l'emplacement géographique.

Pouvons nous estimer le niveau de CO2 à partir de la température $T$, l'Humidité $H$ et l'heure de la journée $t$ for en capteur donné? c.a.d trouver la a function $\hat{f}$:
$$CO2 \sim f(T, H, t)$$

- Température - numerique
- Humidity - numerique
- Heure - numérique

Méthode:

- Modèle d'arbre de régression évalué par Erreur de Moyenne Absolue (MAE)

**mean absolute error (MAE)**:
$$\mathrm {MAE} ={\frac {1}{n} \sum _{i=1}^{n}\left|\hat{y}_{i}-y_{i}\right|},$$ where $\hat{y}_{i}$ is the prediction and $y_{i}$ the true value.

-----
Nous convertissons les timestamp en moments de la journée alignés sur chaque 30min (création d'une nouvelle variable)

0:  00:00\
1:  00:30\
2:  01:00\
...\
47: 23:30


In [ ]:
df['timenum'] = df['Timestamp'].dt.hour * 2 + df['Timestamp'].dt.minute // 30

----
## Reconstruire le CO2 à partir de la température, humidité, heure de la journée

Nous allons essayer différent modèles. De manière à ne pas répéter du code nous créons une fonction qui:

- Découpe les données **df** en ensemble d'apprentissage (X_train, Y_train) et un ensemble de test (X_test, Y_test)
- Apprentissage du modèle en utilisant la méthode **model** (argument) sur les données d'apprentissage
- Évaluation du modèle avec les données de test à l'aide de la métrique **measure** (MAE)
- Visualisation du modèle sous forme de serie séquentielle sur l'interval X_train + X_test.

In [ ]:
def train_plot(df, model, measure, split_date='2017-10-25'):
    """
    Apprend, test et affiche
    """
    
    X = df[['T', 'H', 'timenum']]
    y = df['CO2']
    
    # Echantillons d'apprentissage: toutes les données avant split_date
    X_train = X[df['Timestamp'] < split_date]
    y_train = y[df['Timestamp'] < split_date]
    
    # Echantillons de test toutes les données après the split_date
    X_test = X[df['Timestamp'] >= split_date]
    y_test = y[df['Timestamp'] >= split_date]
    
    # Apprentissage du modèle
    model.fit(X_train, y_train)
    
    # Evaluation du modèle
    y_train_predict = model.predict(X_train)
    train_err = measure(y_train, y_train_predict)
    y_test_predict = model.predict(X_test)
    test_err = measure(y_test, y_test_predict)
    
    print(f'Erreur Aprentissage: {train_err:.2f}, Erreur Test: {test_err:.2f}')
    
    # Affichage
    y_predict = np.append(y_train_predict, y_test_predict)

    plt.subplots(figsize=(20, 5))
    plt.plot(df.Timestamp, y, label = 'Vraie')
    plt.plot(df.Timestamp, y_predict, label = 'Prédite')
    plt.vlines(pd.Timestamp(split_date), ymin=df.CO2.min(), ymax=df.CO2.max(), colors='k', linestyles= 'dashed')
    plt.xlabel("Date")
    plt.ylabel("CO2/ppm")
    plt.legend()
    plt.show()
    
    return model

-----
#### Tentative 1

Utiliser un arbre de regression (DecisionTreeRegressor) pour reconstruire les valeurs de CO2 du capteur WRTW

In [ ]:
df_model = df[df.Location_Name == 'WRTW']

In [ ]:
tree = DecisionTreeRegressor(criterion='mae')
model = train_plot(df_model, tree, mae)

Cet arbre de régression un défaut de sur-apprentissage (Erreur d'apprentissage 0, erreur de prediction importante).

Car nous lui avont permis de grandir sans contrainte et l'algorithm a trouvé une solution très spécifique à la donnée d'apprentissage, mais qui ne se généralise pas aux autres échantillons.

Il est possible de vérifier la forme de l'arbre et de voir qu'il est bien balancé et profond. Pour chaque combinaison $(T,H,t)$ des données d'apprentissage il est sans doute capable de trouver la réponse exacte ou très proche du $CO_2$ correspondante à cette combinaison dans les données d'apprentissage. Mais il n'est pas aussi performant pour les combinaisons de $(T,H,t)$ des données de test, qui lui étaient inconnues au moment de l'apprentissage.

In [ ]:
plt.subplots(figsize=(30,10))
plot_tree(model,max_depth=4,feature_names=['T','H','timenum'])
plt.show()

----
#### Tentative 2

Nous corrigeons le problème de sur-apprentissage avec **un élagage et cross-validation en blocs de 5**:

Pour ce faire, voir les hyperparamètres de [**DecisionTreeRegressor**](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html):

- **max_depth**: profondeur maxinun de l'arbre (c.a.d distance maximum entre le noeud de départ (racine) et les noeuds de terminaison
- **min_samples_split**: le nombre minimum d'échantillons requis pour diviser un noeud interne en sous noeuds.
- **min_samples_leaf**: le nombre minimum de sample requis pour être un noeud de terminaison
- ....

Nous essayons 4 valeurs de max_depth et 5 valeurs de min_samples_split (donc $4*5=20$ combinaisons possible).

De manière à automatiser la recherche nous [**GridSearchCV**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html), qui va essayer les 20 combinaisons possibles d'hyperparamètres pour apprendre des modèles de regression arborescents et les comparer.

La méthode retourne parmis ces modèles celui qui la meilleur performance, en utilisant la métrique de l'argument **scoring**. Nous utilisons **neg_mean_absloute_error** ($-MAE$), car GridSearchCV essaye d'augmenter cette métrique alors que nous voulons la diminuer.

GridSearchCV valide les modèles qu'il obtient avec la méthode de Validation Croisée. L'argument **cv** est le nombre de blocs utilisés pour la validation.

**NB 1**: Par défaut les échantillons sont répartis aléatoirement entre les 5 blocs (donc le résultat peut changer à chaque invocation de GridSearchCV). Il est possible d'utiliser KFold pour imposer une stratégie de selection déterministique.

**NB 2**: Notez que GridSearchCV expose l'interface fit, et predict des modèles. La fonction train_plot ne fait pas la différence entre un objet de type DecisionTreeRegressor ou un objet de type GridSearchCV.


In [ ]:
tree = DecisionTreeRegressor(criterion='mae')

# Grille d'hyperparamètres à utiliser
param_grid = dict(
    max_depth=range(5, 21, 5),          # 5, 10, 15, 20
    min_samples_split=range(10, 51, 10) # 10, 20, 30, 40, 50
)

# Utiliser la Validation Croisée avec MAE
# Attention, GridSearchCV va essayer de maximiser la valeur, nous utilisons
# donc la moyenne d'erreur absolue MAE négative (minimisation)
tree_cv = GridSearchCV(
    tree,
    param_grid,
    cv = 5, # 5 blocs
    #cv = KFold(n_splits=5, shuffle=False),
    scoring='neg_mean_absolute_error'
)

tree_cv=train_plot(df_model, tree_cv, mae)

Nous constatons une amélioration visible de l'erreur de prédiction, ainsi qu'une augmentation de l'erreur d'apprentissage.

Il est possible de vérifier que l'arbre est beaucoup moins complexe que celui appris sans jouer avec les hyperparamètres.

In [ ]:
plt.subplots(figsize=(30,10))
plot_tree(tree_cv.best_estimator_,max_depth=12,feature_names=['T','H','timenum'])
plt.show()

----
## Exercices

Reprendre le premier exemple de la tentative 1 pour construire un modèle de régression avec le type de modèle de [**RandomForestRegressor**](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html) de scikit-learn.



----
## Bonus

Détection de valeurs anormales

In [ ]:
df_kbrl = df[df.Location_Name == 'KBRL']

In [ ]:
df_kbrl.plot('Timestamp', 'CO2', figsize=(20, 5))
plt.annotate('Outlier', size=12,
             xy = ('2017-10-05 07:00:00', 727.537201), 
             xytext = ('2017-10-06 07:00:00', 700), 
             arrowprops=dict(arrowstyle='fancy'))
plt.xlabel("Date")
plt.ylabel("CO2/ppm")
plt.grid(True, which="minor")
plt.show()

In [ ]:
X = df_kbrl[['T', 'H', 'timenum']]
y = df_kbrl['CO2']

In [ ]:
tree = DecisionTreeRegressor(criterion='mae')
tree.fit(X, y)

df_kbrl=df_kbrl.assign(CO2_predict=tree.predict(X))
df_kbrl.plot('Timestamp', ['CO2', 'CO2_predict'], 
             figsize=(20, 5), title = 'Apprentissage aggressif sans validation croisée')
plt.xlabel("Date")
plt.ylabel("CO2/ppm")
plt.grid(True, which="minor")
plt.show()

In [ ]:
plt.subplots(figsize=(30,10))
plot_tree(tree,max_depth=3,feature_names=['T','H','timenum'])
plt.show()

Nous voulons capturer le comportement **général** des niveaux de CO2 mas pas les outliers, donc il est important d'éviter que le sur-apprentissage. La validation croisée s'occupe de ça.

In [ ]:
tree = DecisionTreeRegressor(criterion='mae')

param_grid = dict(
    max_depth=range(5, 21, 5), 
    min_samples_split=range(10, 51, 10)
)

tree_cv = GridSearchCV(
    tree, 
    param_grid, 
    cv = 5,
    scoring='neg_mean_absolute_error'
)

tree_cv.fit(X, y)

df_kbrl = df_kbrl.assign(CO2_predict=tree_cv.predict(X))
df_kbrl.plot('Timestamp', ['CO2', 'CO2_predict'], 
             figsize=(20, 5), title = 'Apprentissage avec validation croisée')
plt.xlabel("Date")
plt.ylabel("CO2/ppm")
plt.grid(True, which="minor")
plt.show()

In [ ]:
plt.subplots(figsize=(30,10))
plot_tree(tree_cv.best_estimator_,max_depth=10,feature_names=['T','H','timenum'])
plt.show()